In [1]:
import pandas as pd
import folium

In [2]:
# https://stackoverflow.com/questions/10944047/how-can-i-remove-a-pytz-timezone-from-a-datetime-object
# https://blog.prototypr.io/interactive-maps-with-python-part-1-aa1563dbe5a9
import arrow
import pandas as pd
bike_data = pd.read_csv("bike_dat_may.csv")
started_hour = []
for i in range(len(bike_data.started_at)):
    arrowObj = arrow.get(bike_data.started_at[i])
    tmpDatetime = arrowObj.datetime
    tmpDatetime = tmpDatetime.replace(tzinfo=None)
    started_hour.append(tmpDatetime.hour)
ended_hour = []
for i in range(len(bike_data.ended_at)):
    arrowObj = arrow.get(bike_data.ended_at[i])
    tmpDatetime = arrowObj.datetime
    tmpDatetime = tmpDatetime.replace(tzinfo=None)
    ended_hour.append(tmpDatetime.hour)

In [3]:
bike_data["started_hour"]=started_hour
bike_data['ended_hour']=ended_hour

In [4]:
def get_trip_counts_by_hour(selected_hour):
    # make a DataFrame with locations for each bike station
    locations = bike_data.groupby("start_station_id").first()
    locations = locations.loc[:, ["start_station_latitude",
                                 "start_station_longitude",
                                 "start_station_name"]]
    
    #select one time of day
    subset_start = bike_data[bike_data["started_hour"]==selected_hour]
    subset_ended = bike_data[bike_data['ended_hour']==selected_hour]
    
    # count trips for each destination
    departure_counts =  subset_start.groupby("start_station_id").count().iloc[:,[0]]
    departure_counts.columns= ["departure_count"]
    
    # count trips for each origin
    arrival_counts =  subset_ended.groupby("end_station_id").count().iloc[:,[0]]
    arrival_counts.columns= ["arrival_count"]

    #join departure counts, arrival counts, and locations
    trip_counts = departure_counts.join(locations).join(arrival_counts)
    return trip_counts

In [5]:
def plot_station_counts(trip_counts):
    # generate a new map
    folium_map = folium.Map(location=[55.953251,-3.188267],
                            zoom_start=11,
                            tiles="CartoDB dark_matter",
                            width= '50%')

    # for each row in the data, add a cicle marker
    for index, row in trip_counts.iterrows():
        # calculate net departures
        net_departures = (row["departure_count"]-row["arrival_count"])
        
        # generate the popup message that is shown on click.
        popup_text = "{}<br> total departures: {}<br> total arrivals: {}<br> net departures: {}"
        popup_text = popup_text.format(row["start_station_name"],
                          row["departure_count"],
                          row["arrival_count"],
                          net_departures)
        
        # radius of circles
        radius = abs(net_departures)
        
        # choose the color of the marker
        if net_departures > 0:
            color="#E37222" # tangerine
        elif net_departures < 0:            
            color="#0A8A9F" # teal
        else:
            color='transparent'
    
        
        # add marker to the map
        folium.CircleMarker(location=(row["start_station_latitude"],
                                      row["start_station_longitude"]),
                            radius=radius,
                            color=color,
                            popup=popup_text,
                            fill=True).add_to(folium_map)
    return folium_map

In [8]:
# plot net departures at 9AM

trip_counts = get_trip_counts_by_hour(18)
plot_station_counts(trip_counts)